In [20]:
#| echo: false
import pandas as pd
import matplotlib.pyplot as plt
import pydeck as pdk
import matplotlib
import matplotlib.dates as mdates
import glob
import geopandas as gpd
from datetime import datetime
from IPython.display import display_markdown
#plt.style.use('IOM.mplstyle')
pd.set_option('display.max_columns', None)

# Background
Why do this coffeee-break session?

## Why use Python for geospatial analysis?
- Reuseable
- Transparency of methods
- Easier for collaboration
- Wide range of tools
- Can integrate with systems/ create data piplines

## Why Python may not be a good fit
- Less cartographic options
- Steeper learning curve
- Less capable for map making

Stick to tools like ArcGIS or QGIS if your needs are cartographic

## Amazing free resources for geospatial analysis in Python
- [Geo-Python 2022](https://geo-python-site.readthedocs.io/en/latest/) - Intro course by University of Helsinki
- [Introduction to Python for Geographic Data Analysis](https://pythongis.org/) - Comprehensive but not fully complete
- [Geographic Data Science with Python](https://geographicdata.science/book/intro.html) - Advanced topics, spatial weights and regressions etc.
- [Intro to Python GIS](https://automating-gis-processes.github.io/CSC18/course-info/course-info.html) - Lots of useful examples, geocoding, zonal stats etc.
- [Isochrone Maps with OSMnx + Python](https://geoffboeing.com/2017/08/isochrone-maps-osmnx-python/)
- [Geopandas docs](https://geopandas.org/en/stable/getting_started/introduction.html) - Cleary explains the concepts and has a comprehensive examples gallery

## Useful tools
- Geopandas - tabular vector data manipulation and analysis
- RasterIO & RasterStats - raster data manipulation and analysis 
- Matplotlib - static plots of vector and raster data
- Pydeck - interactive mapping
- Python-OSRM - routing
- OSMnx - Isochrones
- h3py - Hexagonal spatial indexing

# Geopandas
![](images/geopandas1.png)

- Reading and writing vector data
- Geo data manipulation
- Geometric manipulations

# RasterIO & RasterStats
![](images/worldpop.jpeg)

# Matplotlib
- Built in to geopandas
- Straight forward to plot layers, vectopr and raster
- Labelling is possible but is not elegant
- Functional, quick (just a few lines of code), but with a steep learning curve for anything more complex

# Pydeck

In [21]:
"""
HexagonLayer
==============

Personal injury road accidents in GB from 1979.

The layer aggregates data within the boundary of each hexagon cell.

This example is adapted from the deck.gl documentation.
"""

import pydeck as pdk

HEXAGON_LAYER_DATA = (
    "https://raw.githubusercontent.com/visgl/deck.gl-data/master/examples/3d-heatmap/heatmap-data.csv"  # noqa
)

# Define a layer to display on a map
layer = pdk.Layer(
    "HexagonLayer",
    HEXAGON_LAYER_DATA,
    get_position=["lng", "lat"],
    auto_highlight=True,
    elevation_scale=50,
    pickable=True,
    elevation_range=[0, 3000],
    extruded=True,
    coverage=1,
)

# Set the viewport location
view_state = pdk.ViewState(
    longitude=-1.415, latitude=52.2323, zoom=6, min_zoom=5, max_zoom=15, pitch=40.5, bearing=-27.36,
)

# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html("hexagon_layer.html")

# Python-OSRM or Google Maps API
![](images/osrm.png)

- find the shortest path and travel times/distances between points
- Compute the times between all pairs of points (such as all IDP sites and health facilities)

# OSMnx
![](images/osmnx.webp)

# h3py
<iframe src="https://wolf-h3-viewer.glitch.me/" title="H3" style="width:100%; height:500px"></iframe>

# Example 1 - Hurricane exposure analysis

In [22]:
import requests
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterstats
from rasterstats import zonal_stats, point_query
import shapely
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings("ignore", category=ShapelyDeprecationWarning) 

affectedarea = 'AbacoFloodExtent.shp'
populationraster = 'Worldpop/bhs_ppp_2019.tif'

statsmax = zonal_stats(affectedarea, populationraster , stats="count", all_touched=True)
resultmax = {}
for myDict in statsmax:
    for key, value in myDict.items():
        resultmax.setdefault(key, 0)
        resultmax[key] += value
statsmin = zonal_stats(affectedarea, populationraster , stats="count", all_touched=False)
resultmin = {}
for myDict in statsmax:
    for key, value in myDict.items():
        resultmin.setdefault(key, 0)
        resultmin[key] += value
        
print("Estimated exposed population: between", "{:,}".format(resultmin["count"]),"and","{:,}".format(resultmax["count"]))

Estimated exposed population: between 32,751 and 32,751


# Example 2 - Travel distance Ethiopia

In [23]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# import matplotlib
# matplotlib.rcParams.update({'font.size': 14})
# %matplotlib inline

# #openrouteservice

# def distance(start, end):
#     api_key ='5b3ce3597851110001cf6248a141c1919e684c33a2bec1ab18ea3f4c'
#     url ='https://api.openrouteservice.org/v2/directions/driving-car?'
#     try:
#         r = requests.get(url + 'api_key=' + api_key + '&start=' + start + '&end=' + end)
#         x = r.json()
#         distance = x['features'][0]['properties']['summary']['distance']/1000
#     except KeyError:
#         distance = 'error'
#     return distance

# raw = pd.read_excel('distances.xlsx')
# df = raw
# df = raw[['1.1.f.1: GPS: Longitude', '1.1.f.2: GPS: Latitude','Origin of Large IDP_Longitude','Origin of Large IDP_Latitude']]
# df = df.rename(columns={'1.1.f.1: GPS: Longitude':'current long','1.1.f.2: GPS: Latitude':'current lat','Origin of Large IDP_Longitude':'origin long','Origin of Large IDP_Latitude':'origin lat'})
# df = df.astype(str)
# df['current']= df['current long']+', '+df['current lat']
# df['origin']= df['origin long']+', '+df['origin lat']
# df['distance'] = df.apply(lambda x: distance(x['current'],x['origin']),axis=1)
# df.to_excel('distances-calculated-openrouteservice.xlsx')
# df['distance'].values()

# x = df[df['distance']!='error']['distance'].astype(int)

# fig,ax = plt.subplots(1)
# plt.hist(x, density=True, bins=30)  # density=False would make counts
# plt.ylabel('Frequency')
# plt.xlabel('Distance (km)')
# #ax.set_yticklabels([])
# fig.savefig("distance.png", format="png", transparent=True, bbox_inches='tight', pad_inches=0) 

# Example 3 = H3 in Pakistan
[Pakistan example of survey locations](h3_hexagon_layer.html)